<a href="https://colab.research.google.com/github/vedikajain2004/mlproject/blob/main/OptAlpha_Candle_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import numpy as np
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/s1.csv')
data.head()

,date,time,open,high,low,close,volume,open_interest
0,2017-01-02,09:15:00,100.00,100.28,100.00,100.02,7000.0,100000.0
1,2017-01-02,09:16:00,100.02,100.07,100.02,100.06,3509.0,101212.0
2,2017-01-02,09:17:00,100.05,100.06,99.70,99.73,5105.0,101212.0
3,2017-01-02,09:18:00,99.71,99.79,99.63,99.79,4959.0,101212.0
4,2017-01-02,09:19:00,99.78,99.82,99.69,99.70,2398.0,102480.0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 673384 entries, 0 to 673383
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   date           673384 non-null  object 
 1   time           673384 non-null  object 
 2   open           673384 non-null  float64
 3   high           673384 non-null  float64
 4   low            673384 non-null  float64
 5   close          673384 non-null  float64
 6   volume         673384 non-null  float64
 7   open_interest  673384 non-null  float64
dtypes: float64(6), object(2)
memory usage: 41.1+ MB


In [ ]:
data.describe()

,open,high,low,close,volume,open_interest
count,673384.000000,673384.000000,673384.000000,673384.000000,673384.000000,673384.000000
mean,176.523172,176.580113,176.466306,176.523310,1474.010413,101125.616752
std,44.845303,44.850573,44.839592,44.845318,1896.214092,37082.227375
min,88.270000,88.760000,87.980000,88.280000,0.000000,0.000000
25%,139.490000,139.530000,139.460000,139.490000,446.000000,82293.000000
50%,169.290000,169.350000,169.230000,169.290000,892.000000,98964.000000
75%,210.530000,210.600000,210.460000,210.530000,1795.000000,117471.000000
max,274.030000,274.170000,273.910000,274.030000,293657.000000,908767.000000


In [ ]:
df = pd.DataFrame({'datetime': pd.to_datetime(data['date'] + ' ' + data['time']),
                   'day': pd.to_datetime(data['date'] + ' ' + data['time']).dt.dayofweek,
                   'time': data['time'].str[:2],
                   'oc': data['close'] - data['open'],
                   'hl': data['high'] - data['low'],
                   'vol': data['volume'],
                   'opin': data['open_interest']})
#another column 'colour' would be added once the threshold for neutrality is decided

In [ ]:
n = 10
df['ema'] = np.zeros(len(df))
dates = data['date'].unique()
df['ema'] = data['close'].ewm(span=n, adjust=False).mean()

In [ ]:
df['opnorm'] = data['open'] - df['ema']
df['hinorm'] = data['high'] - df['ema']
df['lonorm'] = data['low'] - df['ema']
df['clnorm'] = data['close'] - df['ema']
df.tail(15)

,datetime,day,time,oc,hl,vol,opin,ema,opnorm,hinorm,lonorm,clnorm
673369,2024-05-24 15:15:00,4,15,-0.04,0.23,3688.0,111354.0,269.422920,0.007080,0.087080,-0.142920,-0.032920
673370,2024-05-24 15:16:00,4,15,0.00,0.10,1243.0,111900.0,269.426025,0.013975,0.083975,-0.016025,0.013975
673371,2024-05-24 15:17:00,4,15,-0.04,0.11,544.0,111900.0,269.417657,0.002343,0.062343,-0.047657,-0.037657
673372,2024-05-24 15:18:00,4,15,-0.03,0.13,934.0,111900.0,269.405356,-0.025356,0.024644,-0.105356,-0.055356
673373,2024-05-24 15:19:00,4,15,0.06,0.11,866.0,111995.0,269.408018,-0.048018,0.051982,-0.058018,0.011982
673374,2024-05-24 15:20:00,4,15,0.04,0.07,835.0,111995.0,269.417470,0.002530,0.052530,-0.017470,0.042530
673375,2024-05-24 15:21:00,4,15,-0.01,0.06,792.0,111995.0,269.423384,0.036616,0.046616,-0.013384,0.026616
673376,2024-05-24 15:22:00,4,15,-0.04,0.09,590.0,111655.0,269.420951,0.029049,0.029049,-0.060951,-0.010951
673377,2024-05-24 15:23:00,4,15,0.01,0.04,827.0,111655.0,269.417141,-0.027141,-0.017141,-0.057141,-0.017141
673378,2024-05-24 15:24:00,4,15,-0.03,0.07,2121.0,111655.0,269.408570,-0.008570,0.021430,-0.048570,-0.038570


In [ ]:
#data subsets for checking clusters and running EDA
usedata = data.tail(60000)
usedf = df.tail(60000)

In [ ]:
#collapsing plots not needed now
#they affected performance negatively
'''import plotly.graph_objects as go
fig = go.Figure(data=[go.Candlestick(x=usedf['datetime'],
                open=usedata['open'],
                high=usedata['high'],
                low=usedata['low'],
                close=usedata['close'])])
fig.show()'''

'''fig = go.Figure(data=[go.Candlestick(x=usedf['datetime'],
                open=usedf['opnorm'],
                high=usedf['hinorm'],
                low=usedf['lonorm'],
                close=usedf['clnorm'])])
fig.show()'''

"fig = go.Figure(data=[go.Candlestick(x=usedf['datetime'],\n                open=usedf['opnorm'],\n                high=usedf['hinorm'],\n                low=usedf['lonorm'],\n                close=usedf['clnorm'])])\nfig.show()"

In [ ]:
#scatter plots

'''plt.scatter(usedf['day'], usedf['oc'])
plt.show()'''
#tuesday had more spikes and falls, saturday is latent, more fluctuations are at the start and end of work week

'''sns.boxplot(x='time', y='oc', data=usedf)
plt.xlabel('time')
plt.ylabel('C-O')
plt.title('Boxplot with Categories on X-axis')
plt.show()
plt.show()'''
#high volatility in first two hours and the last hour before market closing, tranquil at the closing and during mealtime
#volatility generally sees a slowing trend

'''plt.scatter(usedata['open_interest'], usedf['oc'])
plt.show()'''
#needs more analysis

'''plt.scatter(usedf['oc'], usedf['vol'])
plt.show()'''
#this relationship needs more analysis

'''plt.scatter(usedf['day'], usedf['vol'])
plt.show()'''
#trading goes slow on saturday
#tuesday, which had been found to be volatile, has fewer transactions
#wednesday was tranquil, but has volume spikes
#wednesday is the day for prudent trading, as fluctuations are low and volume is high

'''plt.scatter(df['time'].str[:2], df['vol'])
plt.show()'''
#volume slumps in midday, and reduces at the closing time

'''plt.scatter(usedf['vol'], usedata['open_interest'])
plt.show()'''
#more analysis is required

'''plt.scatter(usedf['ema'], usedf['oc'])
plt.show()'''
#more clarity is needed

'''plt.scatter(df['oc'], df['hl'])
plt.show()'''
#as expected, high hl leads to high oc magnitude

'''plt.scatter(usedf['lonorm'], usedf['oc'])
plt.show()'''
#obvious: oc and clnorm are linearly related, oc and opnorm are inversely related
#+ve hinorm and -ve lonorm usually lead to +ve oc

'''plt.scatter(usedf['ema'], usedf['vol'])
plt.show()'''
#more analysis is needed

'''plt.plot(usedf['ema'], usedata['open_interest'])
plt.show()'''
#caught off guard, needs to be scrutinised

'''plt.scatter(usedf['hinorm'], usedf['clnorm'])
plt.show()'''
#OHCL relations: nothing interesting at the first glance, may become useful later

"plt.scatter(usedf['hinorm'], usedf['clnorm'])\nplt.show()"

In [ ]:
#using oc as a check for candle colour
cdf = usedf[['ema', 'oc', 'vol', 'opin']]
kmeans = KMeans(n_clusters=3)
kmeans.fit(cdf)
labels = kmeans.predict(cdf)
u_labels = np.unique(labels)
pcdf = PCA(2).fit_transform(cdf)
'''for i in u_labels:
    plt.scatter(pcdf[labels == i, 0], pcdf[labels == i, 1], label = i)
plt.legend()
plt.show()'''

'for i in u_labels:\n    plt.scatter(pcdf[labels == i, 0], pcdf[labels == i, 1], label = i)\nplt.legend()\nplt.show()'

In [ ]:
kmeans.cluster_centers_

array([[ 2.53378284e+02,  6.55158600e-04,  9.13886376e+02,
         1.15929217e+05],
       [ 2.58174460e+02, -4.99681984e-04,  8.83714382e+02,
         9.24090957e+04],
       [ 2.52564169e+02,  8.92349357e-04,  1.29009045e+03,
         1.51746904e+05]])

In [ ]:
#distribution plots

'''mean = np.mean(usedf[['day', 'oc']])
std = np.std(usedf[['day', 'oc']])
pdf = stats.norm.pdf(usedf[['day', 'oc']], mean, std)
plt.plot(usedf[['day', 'oc']], pdf)
plt.show()'''
#usedf[['day', 'oc']].plot.kde()
'''k = stats.gaussian_kde(usedf['day'], 'oc']])
plt.scatter(usedf[['day', 'oc']], k)
plt.show()'''
#sns.displot(usedf, x='day', y='oc')

#sns.displot(usedf, x='time', y='oc')
#sns.displot(usedf, x='opin', y='oc')
#sns.displot(usedf, x='vol', y='oc')
#sns.displot(usedf, x='day', y='vol')
#sns.displot(usedf, x='time', y='vol')
#sns.displot(usedf, x='opin', y='vol')
#sns.displot(usedf, x='ema', y='oc')
#sns.displot(usedf, x='ema', y='vol')
#sns.displot(usedf, x='ema', y='opin')
#sns.displot(df, x='ema', y='opin')

"k = stats.gaussian_kde(usedf['day'], 'oc']])\nplt.scatter(usedf[['day', 'oc']], k)\nplt.show()"

In [ ]:
df[['ema_1st_grad', 'ema_2nd_grad', 'vol_1st_grad', 'vol_2nd_grad', 'vol_3rd_grad', 'opin_1st_grad', 'opin_2nd_grad']] = np.zeros((len(df), 7))

df['ema_1st_grad'] = df['ema'].rolling(window=n, min_periods=1).apply(lambda x: np.gradient(x)[-1] if len(x) > 1 else np.array([x]), raw=True)

df['ema_2nd_grad'] = df['ema_1st_grad'].rolling(window=n, min_periods=1).apply(lambda x: np.gradient(x)[-1] if len(x) > 1 else np.array([x]), raw=True)

df['vol_1st_grad'] = df['vol'].rolling(window=n, min_periods=1).apply(lambda x: np.gradient(x)[-1] if len(x) > 1 else np.array([x]), raw=True)

df['vol_2nd_grad'] = df['vol_1st_grad'].rolling(window=n, min_periods=1).apply(lambda x: np.gradient(x)[-1] if len(x) > 1 else np.array([x]), raw=True)

df['vol_3rd_grad'] = df['vol_2nd_grad'].rolling(window=n, min_periods=1).apply(lambda x: np.gradient(x)[-1] if len(x) > 1 else np.array([x]), raw=True)

df['opin_1st_grad'] = df['opin'].rolling(window=n, min_periods=1).apply(lambda x: np.gradient(x)[-1] if len(x) > 1 else np.array([x]), raw=True)

df['opin_2nd_grad'] = df['opin_1st_grad'].rolling(window=n, min_periods=1).apply(lambda x: np.gradient(x)[-1] if len(x) > 1 else np.array([x]), raw=True)

In [ ]:
df.tail(15)

,datetime,day,time,oc,hl,vol,opin,ema,opnorm,hinorm,lonorm,clnorm,ema_1st_grad,ema_2nd_grad,vol_1st_grad,vol_2nd_grad,vol_3rd_grad,opin_1st_grad,opin_2nd_grad
673369,2024-05-24 15:15:00,4,15,-0.04,0.23,3688.0,111354.0,269.422920,0.007080,0.087080,-0.142920,-0.032920,-0.007316,-0.011708,2556.0,2876.0,3580.0,0.0,0.0
673370,2024-05-24 15:16:00,4,15,0.00,0.10,1243.0,111900.0,269.426025,0.013975,0.083975,-0.016025,0.013975,0.003105,0.010421,-2445.0,-5001.0,-7877.0,546.0,546.0
673371,2024-05-24 15:17:00,4,15,-0.04,0.11,544.0,111900.0,269.417657,0.002343,0.062343,-0.047657,-0.037657,-0.008368,-0.011474,-699.0,1746.0,6747.0,0.0,-546.0
673372,2024-05-24 15:18:00,4,15,-0.03,0.13,934.0,111900.0,269.405356,-0.025356,0.024644,-0.105356,-0.055356,-0.012301,-0.003933,390.0,1089.0,-657.0,0.0,0.0
673373,2024-05-24 15:19:00,4,15,0.06,0.11,866.0,111995.0,269.408018,-0.048018,0.051982,-0.058018,0.011982,0.002663,0.014964,-68.0,-458.0,-1547.0,95.0,95.0
673374,2024-05-24 15:20:00,4,15,0.04,0.07,835.0,111995.0,269.417470,0.002530,0.052530,-0.017470,0.042530,0.009451,0.006789,-31.0,37.0,495.0,0.0,-95.0
673375,2024-05-24 15:21:00,4,15,-0.01,0.06,792.0,111995.0,269.423384,0.036616,0.046616,-0.013384,0.026616,0.005915,-0.003537,-43.0,-12.0,-49.0,0.0,0.0
673376,2024-05-24 15:22:00,4,15,-0.04,0.09,590.0,111655.0,269.420951,0.029049,0.029049,-0.060951,-0.010951,-0.002433,-0.008348,-202.0,-159.0,-147.0,-340.0,-340.0
673377,2024-05-24 15:23:00,4,15,0.01,0.04,827.0,111655.0,269.417141,-0.027141,-0.017141,-0.057141,-0.017141,-0.003809,-0.001376,237.0,439.0,598.0,0.0,340.0
673378,2024-05-24 15:24:00,4,15,-0.03,0.07,2121.0,111655.0,269.408570,-0.008570,0.021430,-0.048570,-0.038570,-0.008571,-0.004762,1294.0,1057.0,618.0,0.0,0.0


In [ ]:
df[['sma', 'vwap', 'macd-signal', '50sma-200sma', 'ultimate_oscillator', 'disparity_index', 'obv', 'a/d', 'rsi', 'adx', 'vol_freq', 'hi_freq', 'lo_freq']] = np.zeros((len(df), 13))

df['sma'] = data['close'].rolling(window=n, min_periods=1).mean()

cvp = (data['volume'] * data['close']).cumsum()
cv = data['volume'].cumsum()
df['vwap'] = cvp / cv

df['disparity_index'] = ((data['close'] - df['sma']) / df['sma']) * 100

clv = ((data['close'] - data['low']) - (data['high'] - data['close'])) / (data['high'] - data['low'])
df['a/d'] = (clv * df['vol']).cumsum()

df['vol_freq'] = df['vol'].rolling(window=n, min_periods=1).apply(lambda x: x.value_counts().get(x.iloc[-1], 0))

df['hi_freq'] = data['high'].rolling(window=n, min_periods=1).apply(lambda x: x.value_counts().get(x.iloc[-1], 0))

df['lo_freq'] = data['low'].rolling(window=n, min_periods=1).apply(lambda x: x.value_counts().get(x.iloc[-1], 0))

In [ ]:
macd = data['close'].ewm(span=12, adjust=False).mean() - data['close'].ewm(span=26, adjust=False).mean()
df['macd-signal'] = macd - macd.ewm(span=9, adjust=False).mean()

In [ ]:
sma50 = data['close'].rolling(window=50, min_periods=1).mean()
sma200 = data['close'].rolling(window=200, min_periods=1).mean()
df['50sma-200sma'] = sma50 - sma200

In [ ]:
priorclose = data['close'].shift(1)
bp = data['close'] - pd.DataFrame([data['low'], priorclose]).min(axis=1)
tr = pd.DataFrame([data['high'], priorclose]).max(axis=1) - pd.DataFrame([data['low'], priorclose]).min(axis=1)
avg1 = bp.rolling(window=7, min_periods=1).sum() / tr.rolling(window=7, min_periods=1).sum()
avg2 = bp.rolling(window=14, min_periods=1).sum() / tr.rolling(window=14, min_periods=1).sum()
avg3 = bp.rolling(window=28, min_periods=1).sum() / tr.rolling(window=28, min_periods=1).sum()
df['ultimate_oscillator'] = 100 * (4 * avg1 + 2 * avg2 + avg3) / (4 + 2 + 1)

In [ ]:
pc = data['close'].diff()
df.loc[pc > 0, 'obv'] = df['vol']
df.loc[pc < 0, 'obv'] = -df['vol']
df['obv'] = df['obv'].cumsum()

In [ ]:
g = pc.where(pc > 0, 0)
l = pc.where(pc < 0, 0)
ag = g.rolling(window=n, min_periods=1).mean()
al = l.rolling(window=n, min_periods=1).mean()
rs = ag / al
df['rsi'] = 100 - (100 / (1 + rs))

In [ ]:
hicl = (data['high'] - data['close'].shift(1)).abs()
locl = (data['low'] - data['close'].shift(1)).abs()
tr = pd.DataFrame([df['hl'], hicl, locl]).max(axis=1)
up = data['high'] - data['high'].shift(1)
down = data['low'] - data['low'].shift(1)
plusdm = pd.Series(np.zeros(up.shape))
minusdm = pd.Series(np.zeros(up.shape))
plusdm.loc[(up > down) & (up > 0)] = up
minusdm.loc[(up < down) & (down > 0)] = down
atr = tr.rolling(window=n, min_periods=1).mean()
plusdm = plusdm.rolling(window=n, min_periods=1).mean()
minusdm = minusdm.rolling(window=n, min_periods=1).mean()
plusdi = (plusdm / atr) * 100
minusdi = (minusdm / atr) * 100
dx = (abs(plusdi - minusdi) / abs(plusdi + minusdi)) * 100
df['adx'] = dx.rolling(window=n, min_periods=1).mean()

In [ ]:
df.tail(15)

,datetime,day,time,oc,hl,vol,opin,ema,opnorm,hinorm,...,50sma-200sma,ultimate_oscillator,disparity_index,obv,a/d,rsi,adx,vol_freq,hi_freq,lo_freq
673369,2024-05-24 15:15:00,4,15,-0.04,0.23,3688.0,111354.0,269.422920,0.007080,0.087080,...,0.41035,NaN,-0.024494,8093836.0,3.507048e+06,172.093023,NaN,1.0,1.0,1.0
673370,2024-05-24 15:16:00,4,15,0.00,0.10,1243.0,111900.0,269.426025,0.013975,0.083975,...,0.40955,NaN,-0.018183,8095079.0,3.506551e+06,193.939394,NaN,1.0,2.0,2.0
673371,2024-05-24 15:17:00,4,15,-0.04,0.11,544.0,111900.0,269.417657,0.002343,0.062343,...,0.40630,NaN,-0.043414,8094535.0,3.506106e+06,562.500000,NaN,1.0,1.0,1.0
673372,2024-05-24 15:18:00,4,15,-0.03,0.13,934.0,111900.0,269.405356,-0.025356,0.024644,...,0.40405,NaN,-0.051579,8093601.0,3.505890e+06,-400.000000,NaN,1.0,1.0,1.0
673373,2024-05-24 15:19:00,4,15,0.06,0.11,866.0,111995.0,269.408018,-0.048018,0.051982,...,0.40190,NaN,-0.022265,8094467.0,3.506126e+06,-344.444444,NaN,1.0,1.0,1.0
673374,2024-05-24 15:20:00,4,15,0.04,0.07,835.0,111995.0,269.417470,0.002530,0.052530,...,0.40215,NaN,0.000000,8095302.0,3.506723e+06,-100.000000,NaN,1.0,1.0,1.0
673375,2024-05-24 15:21:00,4,15,-0.01,0.06,792.0,111995.0,269.423384,0.036616,0.046616,...,0.40280,NaN,0.000742,8094510.0,3.506987e+06,-166.666667,NaN,1.0,2.0,2.0
673376,2024-05-24 15:22:00,4,15,-0.04,0.09,590.0,111655.0,269.420951,0.029049,0.029049,...,0.40100,NaN,-0.008908,8093920.0,3.507052e+06,-142.857143,NaN,1.0,1.0,1.0
673377,2024-05-24 15:23:00,4,15,0.01,0.04,827.0,111655.0,269.417141,-0.027141,-0.017141,...,0.40075,NaN,-0.005568,8093093.0,3.507879e+06,-84.210526,NaN,1.0,1.0,2.0
673378,2024-05-24 15:24:00,4,15,-0.03,0.07,2121.0,111655.0,269.408570,-0.008570,0.021430,...,0.40075,NaN,-0.013734,8090972.0,3.506364e+06,-200.000000,NaN,1.0,2.0,3.0


In [ ]:
#train test split
traindata, testdata = train_test_split(data, test_size=0.2, train_size=0.8)
traindf, testdf = train_test_split(df, test_size=0.2, train_size=0.8)

In [ ]:
#saving df to avoid recomputation
df.to_csv('/content/drive/MyDrive/df.csv', index=False)

In [ ]:
data.head()

,date,time,open,high,low,close,volume,open_interest
0,2017-01-02,09:15:00,100.00,100.28,100.00,100.02,7000.0,100000.0
1,2017-01-02,09:16:00,100.02,100.07,100.02,100.06,3509.0,101212.0
2,2017-01-02,09:17:00,100.05,100.06,99.70,99.73,5105.0,101212.0
3,2017-01-02,09:18:00,99.71,99.79,99.63,99.79,4959.0,101212.0
4,2017-01-02,09:19:00,99.78,99.82,99.69,99.70,2398.0,102480.0


In [ ]:
data.date.value_counts()

date
2017-01-02    375
2021-10-28    375
2021-11-16    375
2021-11-15    375
2021-11-12    375
             ... 
2023-11-28    347
2020-03-23    317
2020-03-13    316
2021-01-11    271
2017-07-10    220
Name: count, Length: 1797, dtype: int64

In [ ]:
data.date.value_counts().value_counts().sort_index()

count
220       1
271       1
316       1
317       1
347       1
358       2
365       1
368       1
370       2
371       1
372       3
373       4
374       5
375    1773
Name: count, dtype: int64

In [ ]:
data[data['date'].value_counts().value_counts() == 220]

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).